# CSV aus Timo erstellen

1. Ist-Auswertung -> Leistungsnachweis -> alle Rohdaten
1. 01.01. -> Bis ende Laufender Monat
  * Option: Urlaubs + Krankheitstage
  * Attribute:
    * Projektnummer (P.-Nummer)
    * Projekttyp (WB, NWBZ, …)
1. Auswerten
1. Dann als CSV ($ als Delimiter, gab sonst Probleme)
1. Manuell:
  * Typ ergänzen für Urlaub und Krank
  * “-” Stunden in “0” umwandeln
  * “Jahr”, “Monat”, “KW”, “Tag”, “Wochentag” aus Datum ableiten

In [1]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install openpyxl

import pandas
import requests

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
sep = '$'

def read_csv(filename):
    my_data = pandas.read_csv(filename, sep=sep, index_col=False)

    return my_data

In [3]:
input_filename = 'auswertung.csv'
JSESSIONID = ''
start_date = '2021-03-01'
end_date = '2021-03-31'
output = 'out.xlsx'

In [4]:
import time

unix_now_ms = int(time.time()) * 1000

cookies = {
    'JSESSIONID': JSESSIONID,
    'logoffUrl': 'https%3A//www.timo24.de/members/login/',
}

headers = {
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'de-DE,de;q=0.9,en-US;q=0.8,en;q=0.7',
}

params = (
    ('reportType', '6'),
    ('timestamp', unix_now_ms),
    ('seperator', 'OTHER'),
    ('andere', sep),
    ('format', '1'),
)

response = requests.get('https://836.timo24.de/timo/reportexport', headers=headers, params=params, cookies=cookies)
open(input_filename, 'wb').write(response.content)

443116

In [5]:
as_dict = read_csv(input_filename)

df = as_dict.filter(['Datum', 'Mitarbeiter', 'Vorgang', 'Std.', 'Kunde', 'Projekt', 'Beschreibung', 'P.-Nr.', 'Projekttyp'], axis=1)

In [6]:
df = df.rename(columns={"Std.": "Stunden"})
df.head()

,Datum,Mitarbeiter,Vorgang,Stunden,Kunde,Projekt,Beschreibung,P.-Nr.,Projekttyp
0,04.01.21,"Jirka, Simon",NaN,"8,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN
1,04.01.21,"Müller, Cornelia",NaN,"6,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN
2,04.01.21,"Autermann, Christian",NaN,"8,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN
3,04.01.21,"Eymelt, Nicole",NaN,"3,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN
4,04.01.21,"Remke, Albert",NaN,"6,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN


## Typ ergänzen für Urlaub und Krank


In [7]:
import math

def enhance_type(row):
    if row['Projekt'] == 'Urlaub':
        return 'Urlaub'
    elif row['Projekt'] == 'Krank':
        return 'Krank'
    else:
        return row['Projekttyp']    

df['Typ'] = df.apply(lambda row: enhance_type(row), axis=1)
df[df["Typ"] == "Urlaub"].head()

,Datum,Mitarbeiter,Vorgang,Stunden,Kunde,Projekt,Beschreibung,P.-Nr.,Projekttyp,Typ
0,04.01.21,"Jirka, Simon",NaN,"8,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub
1,04.01.21,"Müller, Cornelia",NaN,"6,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub
2,04.01.21,"Autermann, Christian",NaN,"8,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub
3,04.01.21,"Eymelt, Nicole",NaN,"3,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub
4,04.01.21,"Remke, Albert",NaN,"6,00",NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub


## “-” Stunden in “0” umwandeln



In [8]:
from decimal import Decimal

def enhance_hours(row):
    if row['Stunden'] == '-':
        return 0
    else:
        return Decimal(row['Stunden'].replace(',','.'))

df['Stunden'] = df.apply(lambda row: enhance_hours(row), axis=1)
df["Stunden"] = pandas.to_numeric(df["Stunden"])
df[df["Stunden"] == "-"].head()

,Datum,Mitarbeiter,Vorgang,Stunden,Kunde,Projekt,Beschreibung,P.-Nr.,Projekttyp,Typ


## “Jahr”, “Monat”, “KW”, “Tag”, “Wochentag” aus Datum ableiten


In [9]:
import datetime

def derive_year(row):
    return row["date"].year

def derive_month(row):
    return row["date"].month

def derive_cw(row):
    return row["date"].isocalendar()[1]

def derive_day(row):
    return row["date"].day

def derive_dayofweek(row):
    return row["date"].weekday() + 1

df['date'] = df.apply(lambda row: datetime.datetime.strptime(row["Datum"], '%d.%m.%y'), axis=1)
df['Jahr'] = df.apply(lambda row: derive_year(row), axis=1)
df['Monat'] = df.apply(lambda row: derive_month(row), axis=1)
df['KW'] = df.apply(lambda row: derive_cw(row), axis=1)
df['Tag'] = df.apply(lambda row: derive_day(row), axis=1)
df['Wochentag'] = df.apply(lambda row: derive_dayofweek(row), axis=1)
df = df.drop("date", axis=1)
df = df.rename(columns={"Stunden": "Std."})
df

,Datum,Mitarbeiter,Vorgang,Std.,Kunde,Projekt,Beschreibung,P.-Nr.,Projekttyp,Typ,Jahr,Monat,KW,Tag,Wochentag
0,04.01.21,"Jirka, Simon",NaN,8.00,NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub,2021,1,1,4,1
1,04.01.21,"Müller, Cornelia",NaN,6.00,NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub,2021,1,1,4,1
2,04.01.21,"Autermann, Christian",NaN,8.00,NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub,2021,1,1,4,1
3,04.01.21,"Eymelt, Nicole",NaN,3.00,NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub,2021,1,1,4,1
4,04.01.21,"Remke, Albert",NaN,6.00,NaN,Urlaub,URLAUB am 04.01.2021,NaN,NaN,Urlaub,2021,1,1,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3711,30.04.21,"Zaabalawi, Mhd Sufian",ML - Research,6.00,Bundesministerium für Wirtschaft und Energie (...,BMWi – MariGeoRoute 2020-12,NaN,R15080,NWBZ,NWBZ,2021,4,17,30,5
3712,30.04.21,"Hollmann, Carsten",WP5: COS4CLOUD Services in Practice,6.00,EU-H2020,EU-H2020 - COS4CLOUD,NaN,R15073,NWBZ,NWBZ,2021,4,17,30,5
3713,30.04.21,"Jürrens, Eike Hinderk",IT-Support,6.25,Bundesministerium für Wirtschaft und Energie (...,BMWi – MariGeoRoute 2020-12,NaN,R15080,NWBZ,NWBZ,2021,4,17,30,5
3714,30.04.21,"Autermann, Christian",Projektmitarbeit,6.75,EU-H2020,EU - EMODNET Ingestion II 2019-10,NaN,R25247,WB,WB,2021,4,17,30,5


In [12]:
print(df.dtypes)
df.to_excel(output, index = False, header=True, columns=["Datum", "Mitarbeiter", "P.-Nr.", "Projekt", "Vorgang", "Beschreibung", "Projekttyp", "Std.", "Jahr", "Monat", "KW", "Tag", "Wochentag", "Typ"], float_format="%.3f")

Datum            object
Mitarbeiter      object
Vorgang          object
Std.            float64
Kunde            object
Projekt          object
Beschreibung     object
P.-Nr.           object
Projekttyp       object
Typ              object
Jahr              int64
Monat             int64
KW                int64
Tag               int64
Wochentag         int64
dtype: object


# Kontrolle

Stunden (Output nächste Zelle) mit Timo-Auswertung abgleichen

In [13]:
df['Std.'].sum()

11113.04